<a href="https://colab.research.google.com/github/Medalcode/Colab1/blob/main/Eval3Mineria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
import numpy as np
import io
import base64
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.stats import skew, kurtosis
import os
from io import BytesIO

In [43]:
# Configuración inicial para Matplotlib/Seaborn
sns.set_theme(style="whitegrid")


In [44]:

# Variables globales para almacenar el DataFrame y el historial de operaciones (Log)
estado_df = None
entradas_log = []

In [45]:
def get_columnas_numericas(df_entrada):
    """Función auxiliar para obtener columnas numéricas."""
    if df_entrada is not None:
        return df_entrada.select_dtypes(include=np.number).columns.tolist()
    return []


In [46]:
def cargar_datos(archivo_obj, delimitador_elegido):
    """I.1 cargar_datos: Carga el archivo subido en un DataFrame."""
    global estado_df, entradas_log
    entradas_log = [] # Reiniciar el log en cada nueva carga

    if archivo_obj is None:
        return None, "Error: Debe subir un archivo.", None

    ruta_archivo = archivo_obj.name

    try:
        if ruta_archivo.endswith('.csv'):
            if delimitador_elegido == "Coma (,)":
                delimitador = ','
            elif delimitador_elegido == "Punto y Coma (;)":
                delimitador = ';'
            else:
                delimitador = ',' # Delimitador por defecto

            # Intentar leer el archivo con codificaciones comunes
            try:
                df_datos = pd.read_csv(ruta_archivo, delimiter=delimitador, encoding='utf-8')
            except UnicodeDecodeError:
                df_datos = pd.read_csv(ruta_archivo, delimiter=delimitador, encoding='ISO-8859-1')

        elif ruta_archivo.endswith(('.xls', '.xlsx')):
            df_datos = pd.read_excel(ruta_archivo)

        else:
            return None, "Error: Archivo no válido. Suba un archivo CSV o Excel.", None

        # Validación de tipos de datos (debe contener categóricos y numéricos)
        columnas_numericas = df_datos.select_dtypes(include=np.number).columns
        columnas_categoricas = df_datos.select_dtypes(include=['object', 'category']).columns

        if len(columnas_numericas) == 0 or len(columnas_categoricas) == 0:
            mensaje = "Advertencia: El archivo debe contener tanto datos categóricos como numéricos para un análisis completo."
        else:
            mensaje = f"Archivo cargado. Dimensiones: {df_datos.shape}. {len(columnas_numericas)} numéricas, {len(columnas_categoricas)} categóricas."

        estado_df = df_datos
        entradas_log.append(f"Carga: Archivo {os.path.basename(ruta_archivo)} cargado. Se detectaron {df_datos.shape[0]} filas.")

        # Retorna el DataFrame, el mensaje y una vista previa
        return estado_df, mensaje, gr.Dataframe(value=df_datos.head())

    except Exception as e:
        estado_df = None
        return None, f"Error de lectura: {str(e)}", None

In [47]:
def manejar_valores_nulos(df_entrada, nombres_columnas_str, metodo):
    """II.1 manejar_valores_nulos: Manejo interactivo de valores nulos."""
    global estado_df, entradas_log
    df_procesado = df_entrada.copy() if df_entrada is not None else None

    if df_procesado is None:
        return None, "Error: Primero cargue un archivo.", None

    nombres_columnas = [c.strip() for c in nombres_columnas_str.split(",") if c.strip()]
    if not nombres_columnas:
        # Si el usuario no especificó columnas, aplicar a todas las columnas numéricas
        nombres_columnas = get_columnas_numericas(df_procesado)
        if not nombres_columnas:
            return estado_df, "Advertencia: No hay columnas numéricas para aplicar la limpieza de nulos.", gr.Dataframe(value=estado_df.head())


    total_nulos_previos = df_procesado[nombres_columnas].isnull().sum().sum()
    filas_afectadas = 0

    if total_nulos_previos == 0:
        mensaje = "No se encontraron valores nulos en las columnas seleccionadas. No se realizó ninguna operación."
        return estado_df, mensaje, gr.Dataframe(value=estado_df.head())

    if metodo == "Eliminar filas":
        total_filas_originales = len(df_procesado)
        df_procesado = df_procesado.dropna(subset=nombres_columnas)
        filas_afectadas = total_filas_originales - len(df_procesado)
        entradas_log.append(f"Limpieza Nulos: Se eliminaron {filas_afectadas} filas con nulos en {', '.join(nombres_columnas)}.")

    else:
        for col in nombres_columnas:
            if col not in df_procesado.columns or col not in get_columnas_numericas(df_procesado): # Se corrigió aquí también
                entradas_log.append(f"Advertencia: La columna '{col}' no es numérica o no existe para imputación. Se omitió.")
                continue

            valor_imputacion = 0
            if metodo == "Llenar con promedio":
                valor_imputacion = df_procesado[col].mean()
            elif metodo == "Llenar con máximo":
                valor_imputacion = df_procesado[col].max()
            elif metodo == "Llenar con mínimo":
                valor_imputacion = df_procesado[col].min()
            elif metodo == "Llenar con cero":
                valor_imputacion = 0

            nulos_en_columna = df_procesado[col].isnull().sum()
            df_procesado[col] = df_procesado[col].fillna(valor_imputacion)
            filas_afectadas += nulos_en_columna # Suma los nulos que realmente se llenaron
            entradas_log.append(f"Limpieza Nulos: La columna '{col}' se imputó con {metodo.split(' ')[-1]} ({valor_imputacion:.2f}).")

    estado_df = df_procesado
    mensaje = f"Limpieza completada. {total_nulos_previos} valores nulos tratados. Registros afectados: {filas_afectadas}."
    return estado_df, mensaje, gr.Dataframe(value=estado_df.head())

In [48]:
def aplicar_escalado(df_entrada, nombres_columnas_str, metodo_escalado):
    """II.2 aplicar_escalado: Aplica normalización o estandarización."""
    global estado_df, entradas_log
    df_escalado = df_entrada.copy() if df_entrada is not None else None

    if df_escalado is None:
        return None, "Error: Primero cargue un archivo.", None

    nombres_columnas = [c.strip() for c in nombres_columnas_str.split(",") if c.strip()]
    if not nombres_columnas:
        return estado_df, "Error: Debe especificar al menos una columna numérica para escalar.", gr.Dataframe(value=estado_df.head())

    if not all(col in df_escalado.columns and col in get_columnas_numericas(df_escalado) for col in nombres_columnas):
        return estado_df, "Error: Verifique que las columnas existan y sean numéricas.", gr.Dataframe(value=estado_df.head())

    escalador = None
    justificacion_escalado = ""
    if metodo_escalado == "Min-Max":
        escalador = MinMaxScaler()
        justificacion_escalado = "**Recomendación Min-Max:** Se recomienda para algoritmos que esperan un rango acotado (ej. Redes Neuronales) o cuando la distribución no es gaussiana. Sin embargo, es sensible a los *outliers* [10-12]."
    elif metodo_escalado == "Z-Score":
        escalador = StandardScaler()
        justificacion_escalado = "**Recomendación Z-Score:** Se recomienda para algoritmos basados en distancias (ej. K-Means, KNN) o cuando se asume una distribución aproximadamente normal. Es menos sensible a los *outliers* que Min-Max [10, 12, 13]."
    else:
        return estado_df, "Método de escalado no válido.", gr.Dataframe(value=estado_df.head())

    for col in nombres_columnas:
        df_escalado[col] = escalador.fit_transform(df_escalado[[col]])
        entradas_log.append(f"Escalado: Columna '{col}' escalada usando {metodo_escalado}.")

    estado_df = df_escalado
    mensaje = f"Escalado de {', '.join(nombres_columnas)} completado usando {metodo_escalado}. {justificacion_escalado}"
    return estado_df, mensaje, gr.Dataframe(value=estado_df.head())

In [49]:
def detectar_y_tratar_outliers(df_entrada, nombre_columna, tratamiento):
    """II.3 detectar_y_tratar_outliers: Detección por IQR y tratamiento."""
    global estado_df, entradas_log
    df_outliers_tratado = df_entrada.copy() if df_entrada is not None else None

    if df_outliers_tratado is None:
        return None, "Error: Primero cargue un archivo.", None

    if nombre_columna not in df_outliers_tratado.columns or nombre_columna not in get_columnas_numericas(df_outliers_tratado):
        return estado_df, f"Error: La columna '{nombre_columna}' no existe o no es numérica.", gr.Dataframe(value=estado_df.head())

    cuartil_1 = df_outliers_tratado[nombre_columna].quantile(0.25)
    cuartil_3 = df_outliers_tratado[nombre_columna].quantile(0.75)
    rango_iqr = cuartil_3 - cuartil_1
    limite_inferior_iqr = cuartil_1 - 1.5 * rango_iqr
    limite_superior_iqr = cuartil_3 + 1.5 * rango_iqr

    registros_outliers = df_outliers_tratado[(df_outliers_tratado[nombre_columna] < limite_inferior_iqr) | (df_outliers_tratado[nombre_columna] > limite_superior_iqr)]
    numero_outliers = len(registros_outliers)

    if numero_outliers == 0:
        mensaje = f"No se detectaron *outliers* en la columna '{nombre_columna}' (Método IQR)."
        return estado_df, mensaje, gr.Dataframe(value=estado_df.head())

    if tratamiento == "Eliminar registros":
        df_outliers_tratado = df_outliers_tratado[~((df_outliers_tratado[nombre_columna] < limite_inferior_iqr) | (df_outliers_tratado[nombre_columna] > limite_superior_iqr))]
        entradas_log.append(f"Outliers: Se eliminaron {numero_outliers} *outliers* en '{nombre_columna}'.")
        mensaje = f"Se detectaron y eliminaron {numero_outliers} *outliers* en '{nombre_columna}'. Se eliminaron {numero_outliers} filas."

    elif tratamiento == "Capping (Winsorización)":
        df_outliers_tratado[nombre_columna] = np.where(df_outliers_tratado[nombre_columna] > limite_superior_iqr, limite_superior_iqr, df_outliers_tratado[nombre_columna])
        df_outliers_tratado[nombre_columna] = np.where(df_outliers_tratado[nombre_columna] < limite_inferior_iqr, limite_inferior_iqr, df_outliers_tratado[nombre_columna])
        entradas_log.append(f"Outliers: Se aplicó *capping* a {numero_outliers} *outliers* en '{nombre_columna}'.")
        mensaje = f"Se detectaron {numero_outliers} *outliers* y se aplicó *Capping* (Winsorización) para conservar los registros."

    else: # "Informar"
        mensaje = f"Se detectaron {numero_outliers} *outliers* en '{nombre_columna}'. Se recomienda tratarlos, ya que pueden sesgar la media y la desviación estándar [15, 21]."
        estado_df = df_entrada
        return estado_df, mensaje, gr.Dataframe(value=estado_df.head())

    estado_df = df_outliers_tratado
    return estado_df, mensaje, gr.Dataframe(value=estado_df.head())

In [50]:
def ejecutar_analisis(df_entrada):
    """III.1 ejecutar_analisis: Calcula estadísticas descriptivas, correlación, curtosis y asimetría."""
    global entradas_log
    if df_entrada is None:
        return "Error: Primero cargue y procese el archivo.", None

    df_numerico = df_entrada.select_dtypes(include=np.number)

    if df_numerico.empty:
        return "El DataFrame no contiene columnas numéricas para el análisis estadístico.", None

    estadisticas_descriptivas = df_numerico.describe().T
    matriz_correlacion = df_numerico.corr(method='pearson')

    series_curtosis = df_numerico.apply(kurtosis, fisher=False) # Fisher=False para valor absoluto (Normal=3)
    series_asimetria = df_numerico.apply(skew)

    df_forma_distribucion = pd.DataFrame({
        'Curtosis (Normal ≈ 3)': series_curtosis,
        'Asimetría (Skewness)': series_asimetria
    }).round(3)

    texto_interpretacion = "### Resumen de Interpretación:\n"
    texto_interpretacion += "- **Curtosis:** Los valores > 3 (Leptocúrtica) indican un pico más agudo y colas pesadas, sugiriendo más *outliers* [30].\n"
    texto_interpretacion += "- **Asimetría:** Valores positivos (> 0) indican sesgo a la derecha (media > mediana) [29, 31].\n"
    texto_interpretacion += "- **Correlación:** Los valores cercanos a 1 o -1 en el mapa de calor indican relaciones lineales fuertes entre pares de variables [32].\n"

    entradas_log.append("Análisis Estadístico: Cálculos descriptivos, curtosis y asimetría generados.")

    resumen_analisis_texto = (
        f"{texto_interpretacion}\n\n"
        f"**Estadísticas Descriptivas (Media, Desviación, Cuartiles):**\n{estadisticas_descriptivas.to_markdown()}\n\n"
        f"**Forma de la Distribución (Curtosis y Asimetría):**\n{df_forma_distribucion.to_markdown()}\n"
    )

    return resumen_analisis_texto, matriz_correlacion

In [51]:

def generar_graficos(df_entrada, columna_correlacion_heatmap, columna_distribucion_plot):
    """III.2 generar_graficos: Genera un mapa de calor y un histograma/boxplot."""
    global entradas_log

    if df_entrada is None:
        return None, None, "Error: Primero cargue el archivo."

    df_numerico = df_entrada.select_dtypes(include=np.number)

    if df_numerico.empty:
        return None, None, "Advertencia: No hay columnas numéricas para generar gráficos."

    ruta_plot_correlacion = None
    ruta_plot_distribucion = None

    try:
        plt.figure(figsize=(10, 8))
        sns.heatmap(df_numerico.corr(), annot=True, cmap="coolwarm", fmt=".2f")
        plt.title("Mapa de Calor de Correlaciones (Pearson)")
        ruta_plot_correlacion = "correlation_plot.png"
        plt.savefig(ruta_plot_correlacion)
        plt.close()
        entradas_log.append("Visualización: Mapa de calor de correlaciones generado.")
    except Exception as e:
        entradas_log.append(f"Error al generar mapa de correlación: {e}")


    if columna_distribucion_plot and columna_distribucion_plot in df_numerico.columns:
        try:
            figura, ejes = plt.subplots(2, 1, figsize=(8, 8), sharex=True)

            sns.histplot(df_numerico[columna_distribucion_plot], kde=True, ax=ejes[0])
            ejes[0].set_title(f"Distribución de: {columna_distribucion_plot} (Histograma y KDE)")

            sns.boxplot(x=df_numerico[columna_distribucion_plot], ax=ejes[1])
            ejes[1].set_title(f"Boxplot de: {columna_distribucion_plot} (Outliers: 1.5*IQR)")

            plt.tight_layout()
            ruta_plot_distribucion = "distribution_plot.png"
            plt.savefig(ruta_plot_distribucion)
            plt.close()
            entradas_log.append(f"Visualización: Gráfico de distribución para '{columna_distribucion_plot}' generado.")
        except Exception as e:
            entradas_log.append(f"Error al generar gráfico de distribución para '{columna_distribucion_plot}': {e}")
    else:
        entradas_log.append("Advertencia: No se pudo generar el gráfico de distribución, columna no numérica o inexistente.")

    return ruta_plot_correlacion, ruta_plot_distribucion, "Gráficos generados correctamente." if ruta_plot_correlacion or ruta_plot_distribucion else "No se pudo generar ningún gráfico."



In [52]:

def exportar_resultados(df_entrada, formato_exportacion):
    """IV.1 exportar_resultados: Permite la exportación y genera el reporte de log."""
    global entradas_log

    if df_entrada is None:
        return "Error: No hay datos procesados para exportar.", None, None

    ruta_reporte = "reporte_analisis.txt"
    contenido_log = "\n".join(entradas_log)

    contenido_reporte_final = (
        "### REPORTE BREVE AUTOMÁTICO DE PROCESAMIENTO DE DATOS\n\n"
        "**Proceso Seguido y Decisiones Tomadas en Limpieza de Datos:**\n"
        f"{contenido_log}\n\n"
        f"**Interpretación Preliminar de Resultados Obtenidos:**\n"
        f"(La interpretación completa de correlaciones, curtosis y regresiones debe realizarla el analista.)\n"
        f"Se recomienda revisar el *heatmap* para correlaciones fuertes (Pearson > 0.7 o < -0.7) [32, 38].\n"
        f"La limpieza de datos asegura la calidad y reduce el sesgo en fases de modelado posteriores (GIGO: *Garbage In, Garbage Out*) [39].\n"
        f"Dimensiones del DataFrame final: {df_entrada.shape}\n"
    )

    with open(ruta_reporte, "w") as f:
        f.write(contenido_reporte_final)

    ruta_salida = None
    if formato_exportacion == "CSV":
        ruta_salida = "datos_procesados.csv"
        df_entrada.to_csv(ruta_salida, index=False)
    elif formato_exportacion == "Excel":
        ruta_salida = "datos_procesados.xlsx"
        df_entrada.to_excel(ruta_salida, index=False)
    else:
        return "Error: Formato de exportación no válido.", None, None

    entradas_log.append(f"Exportación: Datos procesados guardados en {ruta_salida} y Log generado.")

    return f"Exportación exitosa. Descargue el archivo y el reporte.", ruta_salida, ruta_reporte


In [53]:
# ----------------- Estructura de la Interfaz Gradio -----------------

with gr.Blocks(title="Aplicación de Minería de Datos y EDA") as interfaz:
    gr.Markdown("## 🛠️ Aplicación Interactiva para Procesamiento y Análisis de Datos")

    estado_df_gradio = gr.State(None)

    with gr.Tab("1. Carga de Datos"):
        gr.Markdown("### Carga y Validación del Archivo")
        with gr.Row():
            radio_separador = gr.Radio(
                choices=["Coma (,)", "Punto y Coma (;)"],
                label="Selecciona el Separador del Archivo",
                value="Coma (,)",
                interactive=True
            )
            input_archivo = gr.File(label="Subir Archivo (CSV o Excel)", interactive=True)

        btn_cargar_datos = gr.Button("Cargar y Validar")
        msg_carga_datos = gr.Textbox(label="Mensaje de Carga")
        df_vista_previa = gr.Dataframe(label="Vista Previa (5 primeras filas)")

        btn_cargar_datos.click(
            fn=cargar_datos,
            inputs=[input_archivo, radio_separador],
            outputs=[estado_df_gradio, msg_carga_datos, df_vista_previa]
        )

    with gr.Tab("2. Procesamiento y Limpieza (Preparación de Datos)"):
        gr.Markdown("### Limpieza de Valores Nulos")
        with gr.Row():
            radio_metodo_nulos = gr.Radio(
                choices=["Eliminar filas", "Llenar con promedio", "Llenar con máximo", "Llenar con mínimo", "Llenar con cero"],
                label="Método para manejar nulos [6, 41]",
                value="Eliminar filas"
            )
            input_col_nulos = gr.Textbox(label="Columnas para Limpieza (Separadas por comas)", placeholder="Ej: Col1, Col2 (Dejar vacío para todo el DF)")

        btn_aplicar_nulos = gr.Button("Aplicar Limpieza de Nulos")
        msg_resultado_nulos = gr.Textbox(label="Resultado Nulos")

        gr.Markdown("### Normalización y Estandarización")
        with gr.Row():
            radio_metodo_escalado = gr.Radio(
                choices=["Min-Max", "Z-Score"],
                label="Método de Escalado [10, 42]",
                value="Z-Score"
            )
            input_col_escalar = gr.Textbox(label="Columnas Numéricas para Escalar (Separadas por comas)", placeholder="Ej: Edad, Salario")

        btn_aplicar_escalado = gr.Button("Aplicar Normalización / Estandarización")
        msg_resultado_escalado = gr.Textbox(label="Resultado Normalización y Justificación [10]")

        gr.Markdown("### Detección y Tratamiento de Outliers (IQR)")
        with gr.Row():
            input_col_outliers = gr.Textbox(label="Columna para Detección de Outliers (Una sola columna)", placeholder="Ej: Ingresos")
            radio_tratamiento_outliers = gr.Radio(
                choices=["Informar", "Eliminar registros", "Capping (Winsorización)"],
                label="Tratamiento de Outliers [10, 19, 20]",
                value="Informar"
            )

        btn_detectar_outliers = gr.Button("Detectar y Tratar Outliers")
        msg_resultado_outliers = gr.Textbox(label="Resultado Outliers")

        btn_aplicar_nulos.click(
            fn=manejar_valores_nulos,
            inputs=[estado_df_gradio, input_col_nulos, radio_metodo_nulos],
            outputs=[estado_df_gradio, msg_resultado_nulos, df_vista_previa]
        )
        btn_aplicar_escalado.click(
            fn=aplicar_escalado,
            inputs=[estado_df_gradio, input_col_escalar, radio_metodo_escalado],
            outputs=[estado_df_gradio, msg_resultado_escalado, df_vista_previa]
        )
        btn_detectar_outliers.click(
            fn=detectar_y_tratar_outliers,
            inputs=[estado_df_gradio, input_col_outliers, radio_tratamiento_outliers],
            outputs=[estado_df_gradio, msg_resultado_outliers, df_vista_previa]
        )

    with gr.Tab("3. Análisis y Visualización"):
        gr.Markdown("### Análisis Estadístico (Correlación, Curtosis y Asimetría) [25]")

        btn_ejecutar_analisis = gr.Button("Ejecutar Análisis Estadístico")
        output_analisis = gr.Markdown(label="Resumen Estadístico e Interpretación")

        gr.Markdown("### Visualización de Datos Procesados [25]")
        with gr.Row():
            input_col_distribucion = gr.Textbox(label="Columna para Gráfico de Distribución (Histograma/Boxplot)", placeholder="Ej: Edad")

        btn_generar_graficos = gr.Button("Generar Gráficos")

        with gr.Row():
            plot_correlacion = gr.Plot(label="Mapa de Calor de Correlaciones")
            plot_distribucion = gr.Plot(label="Distribución y Outliers (Boxplot/Histograma)")
        msg_graficos = gr.Textbox(label="Mensaje de Gráficos")

        btn_ejecutar_analisis.click(
            fn=ejecutar_analisis,
            inputs=[estado_df_gradio],
            outputs=[output_analisis, gr.State(None)]
        )
        btn_generar_graficos.click(
            fn=generar_graficos,
            inputs=[estado_df_gradio, gr.State(None), input_col_distribucion],
            outputs=[plot_correlacion, plot_distribucion, msg_graficos]
        )

    with gr.Tab("4. Exportación y Reporte"):
        gr.Markdown("### Exportar Datos Procesados y Generar Log [36]")

        radio_formato_exportacion = gr.Radio(
            choices=["CSV", "Excel"],
            label="Seleccionar Formato de Exportación",
            value="CSV"
        )

        btn_generar_archivos = gr.Button("Generar Archivos Finales")
        msg_exportacion = gr.Textbox(label="Resultado de la Exportación")

        output_archivo_datos = gr.File(label="Descargar Datos Procesados")
        output_archivo_log = gr.File(label="Descargar Reporte de Log")

        btn_generar_archivos.click(
            fn=exportar_resultados,
            inputs=[estado_df_gradio, radio_formato_exportacion],
            outputs=[msg_exportacion, output_archivo_datos, output_archivo_log]
        )

if __name__ == "__main__":
    interfaz.launch(inline=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7cb3ed577517d23906.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
